<a href="https://colab.research.google.com/github/GuendouziWassila/KNN_BookRecommendationn/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [16]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-01-12 01:23:09--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2024-01-12 01:23:09 (224 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [39]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

'''
# Filter out users with less than 200 ratings
user_ratings_count = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(user_ratings_count[user_ratings_count >= 200].index)]

# Filter out books with less than 100 ratings
book_ratings_count = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[df_ratings['isbn'].isin(book_ratings_count[book_ratings_count >= 100].index)]
'''


print (df_ratings.shape[0])
print (df_ratings.shape[1])


1149780
3


In [40]:
# create the model

user_mapping = {user: idx for idx, user in enumerate(df_ratings['user'].unique())}
isbn_mapping = {isbn: idx for idx, isbn in enumerate(df_ratings['isbn'].unique())}

# Create a sparse matrix from the data
num_users = len(user_mapping)
num_isbn = len(isbn_mapping)

ratings_matrix = csr_matrix((df_ratings['rating'],
                           (df_ratings['user'].map(user_mapping),
                            df_ratings['isbn'].map(isbn_mapping)))
                          )

# Transpose the matrix to have items in rows and users in columns
ratings_matrix_transposed = ratings_matrix.T
print (ratings_matrix.shape)
print (ratings_matrix_transposed.shape)

# Create a Nearest Neighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)

# Fit the model with the transposed ratings matrix
model_knn.fit(ratings_matrix_transposed)


(105283, 340556)
(340556, 105283)


NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [41]:
# function to return recommended books - this will be tested
def get_recommends(book_title):

    #Given an item, find its nearest neighbors
    book_isbn = df_books.loc[     df_books['title']==book_title, 'isbn'   ]
    if not book_isbn.empty:
      isbn_to_find = book_isbn.values[0]
      print(f"The ISBN for '{book_title}' is {isbn_to_find}")
    else:
      return(f"No book with the title '{book_title}' found.")

    # Find the index of the ISBN in the sparse matrix
    # Perform a reverse lookup in the isbn_mapping dictionary
    index = next((idx for isbn, idx in isbn_mapping.items() if isbn ==  isbn_to_find), None)


    if index is not None:
         print(f"The index for ISBN '{isbn_to_find}' is {index}")
    else:
        print(f"No index found for ISBN '{isbn_to_find}' in the mapping.")


    item_ratings_to_query = ratings_matrix_transposed[index]
    #print (ratings_matrix_transposed[index])
    #item_ratings_to_query = item_ratings_to_query.reshape(1, -1)  # Reshape for compatibility

    # Find neighbors
    distances, indices = model_knn.kneighbors(item_ratings_to_query, n_neighbors=6)
    print (indices[0])

    # Extract book titles and distances from the recommendations
    isbn_list =[]
    for ind in indices[0]:
      # create a list to contain the isbn of the recommanded books
      isbn_list.append(next((isbn for isbn, idx in isbn_mapping.items() if idx == ind), None))

    recommended_books = [
        df_books.loc[df_books['isbn']==isbn,'title'].values[0] for isbn in isbn_list
          ]
    distances_list = distances.flatten()


    # Combine the book title with its recommended books and distances
    result = [book_title, list(zip(recommended_books[1:], distances_list[1:]))]

    return result


In [42]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

The ISBN for 'Where the Heart Is (Oprah's Book Club (Paperback))' is 0446672211
The index for ISBN '0446672211' is 2844
[  2844  23218  36880 101389  53221  34539]


IndexError: index 0 is out of bounds for axis 0 with size 0